In [1]:
# import numpy as np
# import pandas as pd
# import re, nltk, spacy, gensim
# from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD #For LDA
# from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
# from sklearn.model_selection import GridSearchCV
# from pprint import pprint

In [2]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')
import json

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/kacha_nimbu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
stemmer=SnowballStemmer('english')
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))



In [4]:
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [21]:
import ast
file = open('../march15dict.txt', 'r').read()
news_dict = ast.literal_eval(file)
news_list=[]
for key in news_dict:
    news_list.append(news_dict[key])

In [22]:
from pandas import *
df=DataFrame()

In [23]:
df['headline_text']=news_list
df['index'] = range(1, len(df) + 1)
df.head()
len(df)

61

In [24]:
doc_sample = df[df['index'] == 43].values[0][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['Good', 'Friday.', '(Want', 'this', 'by', 'email?', 'Sign', 'up', 'here.)A', 'fight', 'has', 'broken', 'out', 'between', 'TPG', 'and', 'Bill', 'McGlashan,', 'the', 'senior', 'executive', 'caught', 'up', 'in', 'a', 'huge', 'college', 'bribery', 'scandal,', 'over', 'how', 'he', 'left', 'the', '$103', 'billion', 'investment', 'giant,', 'Michael', 'de', 'la', 'Merced', 'of', 'DealBook', 'reports.Mr.', 'McGlashan', 'had', 'initially', 'been', 'put', 'on', 'leave', 'by', 'TPG', 'after', 'being', 'accused', 'of', 'collaborating', 'with', 'William', 'Singer,', 'who', 'ran', 'the', 'college', 'preparatory', 'business', 'at', 'the', 'center', 'of', 'the', 'scandal.', 'Prosecutors', 'say', 'that', 'Mr.', 'Singer', 'advised', 'the', 'financier', 'to', 'claim', 'that', 'his', 'son', 'had', 'learning', 'disabilities,', 'and', 'that', 'Mr.', 'McGlashan', 'paid', '$50,000', 'to', 'Mr.', 'Singers', 'charity.TPG', 'said', 'it', 'fired', 'him', 'for', 'cause', 'yesterday.', 'We', 'be

In [25]:
processed_docs = df['headline_text'].map(preprocess)
processed_docs[:2]

0    [scammer, cheat, paradigmat, figur, presid, un...
1    [post, updat, friday, gunman, strap, helmet, c...
Name: headline_text, dtype: object

In [26]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 accomplish
1 acquaint
2 activ
3 actress
4 admiss
5 adolesc
6 ador
7 adulthood
8 advantag
9 alex
10 amaz


In [27]:
dictionary.filter_extremes(no_below=15, no_above=0.6, keep_n=200000)

In [28]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[43]

[(0, 1),
 (3, 2),
 (4, 1),
 (5, 2),
 (8, 11),
 (9, 2),
 (10, 1),
 (13, 4),
 (14, 1),
 (15, 2),
 (20, 2),
 (23, 3),
 (24, 1),
 (29, 2),
 (31, 4),
 (32, 1),
 (39, 3),
 (46, 1),
 (49, 2),
 (50, 1),
 (52, 1),
 (54, 1),
 (55, 10),
 (63, 1),
 (65, 9),
 (66, 2),
 (67, 1),
 (70, 1),
 (72, 1),
 (73, 2),
 (76, 2),
 (79, 1),
 (80, 1),
 (81, 1),
 (82, 2),
 (83, 1),
 (85, 1),
 (90, 1),
 (92, 2),
 (99, 4),
 (111, 1),
 (112, 3),
 (119, 1),
 (120, 1),
 (121, 1),
 (123, 1),
 (126, 1),
 (129, 3),
 (130, 1),
 (139, 1),
 (140, 1),
 (142, 1),
 (143, 2),
 (145, 2),
 (146, 3),
 (148, 3),
 (149, 2),
 (150, 1),
 (152, 1),
 (160, 1),
 (161, 3),
 (167, 1),
 (170, 1),
 (175, 3),
 (177, 1),
 (179, 1),
 (180, 1),
 (181, 3),
 (183, 1),
 (184, 1),
 (187, 2),
 (190, 1),
 (195, 1),
 (197, 2)]

In [29]:
bow_doc_43 = bow_corpus[43]
for i in range(len(bow_doc_43)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_43[i][0], 
                                               dictionary[bow_doc_43[i][0]], 
bow_doc_43[i][1]))

Word 0 ("base") appears 1 time.
Word 3 ("build") appears 2 time.
Word 4 ("case") appears 1 time.
Word 5 ("charg") appears 2 time.
Word 8 ("compani") appears 11 time.
Word 9 ("cost") appears 2 time.
Word 10 ("countri") appears 1 time.
Word 13 ("execut") appears 4 time.
Word 14 ("expect") appears 1 time.
Word 15 ("face") appears 2 time.
Word 20 ("have") appears 2 time.
Word 23 ("high") appears 3 time.
Word 24 ("learn") appears 1 time.
Word 29 ("make") appears 2 time.
Word 31 ("million") appears 4 time.
Word 32 ("month") appears 1 time.
Word 39 ("plan") appears 3 time.
Word 46 ("rais") appears 1 time.
Word 49 ("recent") appears 2 time.
Word 50 ("report") appears 1 time.
Word 52 ("seven") appears 1 time.
Word 54 ("start") appears 1 time.
Word 55 ("state") appears 10 time.
Word 63 ("turn") appears 1 time.
Word 65 ("unit") appears 9 time.
Word 66 ("week") appears 2 time.
Word 67 ("world") appears 1 time.
Word 70 ("allow") appears 1 time.
Word 72 ("begin") appears 1 time.
Word 73 ("board") ap

In [30]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.06981039542797939),
 (1, 0.13351204075226575),
 (2, 0.15996889880603635),
 (3, 0.0583966520236445),
 (4, 0.04874646947753733),
 (5, 0.06384376177677059),
 (6, 0.061060980515050156),
 (7, 0.45843721753344024),
 (8, 0.07302147211467168),
 (9, 0.08377968837292023),
 (10, 0.2002680611283986),
 (11, 0.08781994632657492),
 (12, 0.07640620292224005),
 (13, 0.06981039542797939),
 (14, 0.08377968837292023),
 (15, 0.05584110248303853),
 (16, 0.06981039542797939),
 (17, 0.10677159358321707),
 (18, 0.04654967318483164),
 (19, 0.15996889880603635),
 (20, 0.07302147211467168),
 (21, 0.10677159358321707),
 (22, 0.1097755864654407),
 (23, 0.33504661489823123),
 (24, 0.06981039542797939),
 (25, 0.04874646947753733),
 (26, 0.07998444940301817),
 (27, 0.10677159358321707),
 (28, 0.13962079085595877),
 (29, 0.04874646947753733),
 (30, 0.08781994632657492),
 (31, 0.12212196103010031),
 (32, 0.04442735907870365),
 (33, 0.0583966520236445),
 (34, 0.08781994632657492),
 (35, 0.08377968837292023),
 (36,

In [31]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=6, id2word=dictionary, passes=2, workers=2)

In [32]:
for idx, topic in lda_model.print_topics(-1):
     print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.043*"compani" + 0.021*"state" + 0.018*"american" + 0.018*"unit" + 0.018*"peopl" + 0.017*"percent" + 0.015*"public" + 0.014*"student" + 0.013*"long" + 0.013*"make"
Topic: 1 
Words: 0.035*"trump" + 0.023*"presid" + 0.015*"tell" + 0.014*"state" + 0.014*"know" + 0.014*"stori" + 0.013*"report" + 0.013*"peopl" + 0.012*"write" + 0.012*"case"
Topic: 2 
Words: 0.031*"peopl" + 0.022*"list" + 0.020*"govern" + 0.020*"includ" + 0.019*"shoot" + 0.015*"report" + 0.014*"offici" + 0.013*"friday" + 0.013*"citi" + 0.012*"famili"
Topic: 3 
Words: 0.045*"school" + 0.019*"million" + 0.015*"go" + 0.015*"offici" + 0.015*"student" + 0.014*"money" + 0.014*"compani" + 0.013*"good" + 0.013*"citi" + 0.013*"includ"
Topic: 4 
Words: 0.027*"school" + 0.025*"need" + 0.024*"colleg" + 0.021*"student" + 0.017*"york" + 0.016*"famili" + 0.013*"help" + 0.013*"long" + 0.013*"peopl" + 0.013*"know"
Topic: 5 
Words: 0.029*"peopl" + 0.021*"want" + 0.020*"york" + 0.015*"know" + 0.015*"citi" + 0.014*"go" + 0.014

In [33]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=6, id2word=dictionary, passes=4, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))
    print('\n')

Topic: 0 Word: 0.018*"trump" + 0.016*"social" + 0.015*"final" + 0.014*"real" + 0.014*"twitter" + 0.012*"life" + 0.011*"nytim" + 0.010*"york" + 0.009*"report" + 0.009*"live"


Topic: 1 Word: 0.012*"depart" + 0.011*"certain" + 0.011*"direct" + 0.010*"social" + 0.007*"compani" + 0.007*"continu" + 0.007*"list" + 0.007*"nation" + 0.007*"final" + 0.007*"director"


Topic: 2 Word: 0.012*"compani" + 0.009*"citi" + 0.008*"famili" + 0.008*"peopl" + 0.008*"state" + 0.008*"trump" + 0.008*"york" + 0.008*"live" + 0.008*"stori" + 0.008*"write"


Topic: 3 Word: 0.031*"trump" + 0.012*"presid" + 0.011*"depart" + 0.008*"accus" + 0.008*"releas" + 0.008*"case" + 0.007*"state" + 0.007*"nation" + 0.007*"friday" + 0.007*"govern"


Topic: 4 Word: 0.023*"student" + 0.018*"school" + 0.014*"colleg" + 0.010*"need" + 0.009*"govern" + 0.008*"compani" + 0.008*"state" + 0.007*"director" + 0.007*"million" + 0.007*"money"


Topic: 5 Word: 0.011*"polit" + 0.010*"trump" + 0.010*"white" + 0.009*"presid" + 0.009*"support" +

In [34]:
unseen_document = 'i love her'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 4)))

Score: 0.5791175961494446	 Topic: 0.027*"school" + 0.025*"need" + 0.024*"colleg" + 0.021*"student"
Score: 0.08475913852453232	 Topic: 0.029*"peopl" + 0.021*"want" + 0.020*"york" + 0.015*"know"
Score: 0.08417338132858276	 Topic: 0.035*"trump" + 0.023*"presid" + 0.015*"tell" + 0.014*"state"
Score: 0.08410275727510452	 Topic: 0.045*"school" + 0.019*"million" + 0.015*"go" + 0.015*"offici"
Score: 0.08405321836471558	 Topic: 0.043*"compani" + 0.021*"state" + 0.018*"american" + 0.018*"unit"
Score: 0.08379385620355606	 Topic: 0.031*"peopl" + 0.022*"list" + 0.020*"govern" + 0.020*"includ"


In [35]:

# Load a potentially pretrained model from disk.
#lda = LdaModel.load(temp_file)
top_words_per_topic=[]
string_sent=""
print(lda_model.num_topics)
for t in range(lda_model.num_topics):
    top_words_per_topic.extend([(t, ) + x for x in lda_model.show_topic(t, topn = 3)])
for t in range(lda_model.num_topics):
    string_sent+=str(top_words_per_topic[t][0])+" "+str(top_words_per_topic[t][1])+" "+str(top_words_per_topic[t][2])
    string_sent+='\n'
    
print(string_sent)  
for idx, topic in lda_model.print_topics(-1):
    print("Topic: "+str(idx))
    print(topic)
    

6
0 compani 0.04338564
0 state 0.02117585
0 american 0.018027103
1 trump 0.035471614
1 presid 0.023229191
1 tell 0.01523344

Topic: 0
0.043*"compani" + 0.021*"state" + 0.018*"american" + 0.018*"unit" + 0.018*"peopl" + 0.017*"percent" + 0.015*"public" + 0.014*"student" + 0.013*"long" + 0.013*"make"
Topic: 1
0.035*"trump" + 0.023*"presid" + 0.015*"tell" + 0.014*"state" + 0.014*"know" + 0.014*"stori" + 0.013*"report" + 0.013*"peopl" + 0.012*"write" + 0.012*"case"
Topic: 2
0.031*"peopl" + 0.022*"list" + 0.020*"govern" + 0.020*"includ" + 0.019*"shoot" + 0.015*"report" + 0.014*"offici" + 0.013*"friday" + 0.013*"citi" + 0.012*"famili"
Topic: 3
0.045*"school" + 0.019*"million" + 0.015*"go" + 0.015*"offici" + 0.015*"student" + 0.014*"money" + 0.014*"compani" + 0.013*"good" + 0.013*"citi" + 0.013*"includ"
Topic: 4
0.027*"school" + 0.025*"need" + 0.024*"colleg" + 0.021*"student" + 0.017*"york" + 0.016*"famili" + 0.013*"help" + 0.013*"long" + 0.013*"peopl" + 0.013*"know"
Topic: 5
0.029*"peopl" + 0

In [36]:
lda_model.save('march15model')